<a href="https://colab.research.google.com/github/tokuton1024/Kaggle/blob/master/baseline_02_Scibert%E3%81%AE%E5%A0%B4%E5%90%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount

In [1]:
from google.colab import drive, files
drive.mount('/content/drive')  # drive をマウント

Mounted at /content/drive


# Setup

In [ ]:
import os
import shutil
from requests import get


! pip install --quiet signate
! pip install --quiet transformers
! pip install --quiet tensorflow-addons

os.makedirs("/root/.signate", exist_ok=True)  # API token 用のディレクトリを作成
token_path = "/content/drive/MyDrive/Signate/signate.json"  # token の path
shutil.copy(token_path, "/root/.signate")  # tokenを"/root/.signate"にコピー

DRIVE = "/content/drive/MyDrive/competitioin/Signate-Systematic-Review"  # Drive のパス
EXP = get('http://172.28.0.2:9000/api/sessions').json()[0]['name'][:-6]  # notebook の名前を取得
INPUT = os.path.join(DRIVE, "input")
OUTPUT = os.path.join(DRIVE, "output")
OUTPUT_EXP = os.path.join(OUTPUT, EXP)
OUTPUT_EXP_MODEL = os.path.join(OUTPUT_EXP, "model")
OUTPUT_EXP_PREDS = os.path.join(OUTPUT_EXP, "preds")
OUTPUT_EXP_FIG = os.path.join(OUTPUT_EXP, "fig")
SUBMISSION = os.path.join(DRIVE, "submission")

if not os.path.isdir(INPUT):  # 初回時にのみデータをINPUTフォルダにロード
    os.makedirs(INPUT, exist_ok=True)
    ! signate download --competition-id=471 --path=$INPUT

# 実験ごとのフォルダを作成
dirs = [INPUT, OUTPUT, OUTPUT_EXP, SUBMISSION, OUTPUT_EXP_MODEL, OUTPUT_EXP_PREDS, OUTPUT_EXP_FIG]
for d in dirs:
    os.makedirs(d, exist_ok=True)

In [ ]:
!pip install -q transformers==2.11.0
!wget https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/tensorflow_models/scibert_scivocab_uncased.tar.gz
!tar -xvf ./scibert_scivocab_uncased.tar.gz

In [ ]:
os.environ["WANDB_API_KEY"] = "0" ## to silence warning
!transformers-cli convert --model_type bert \
  --tf_checkpoint './scibert_scivocab_uncased/bert_model.ckpt' \
  --config './scibert_scivocab_uncased/bert_config.json' \
  --pytorch_dump_output './scibert_scivocab_uncased/pytorch_model.bin'

In [5]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)
import seaborn as sns
import tensorflow as tf

from transformers import BertTokenizer ,BertConfig
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from transformers import TFBertModel
from tensorflow.keras.layers import Dense, Flatten

import time
from transformers import create_optimizer
import transformers

print('transformers version :', transformers.__version__)

transformers version : 2.11.0


In [ ]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .

# TPU Setting

In [29]:
import warnings
warnings.filterwarnings("ignore")  # ignore warnings

# TPUのSetting、
try:
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', TPU.cluster_spec().as_dict()['worker'])
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
except ValueError:
    TPU = None
    print('INFO: Not connected to a TPU runtime')

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  ['10.107.172.106:8470']
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.107.172.106:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.107.172.106:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


# Load Data

In [12]:
# load data
train_data = pd.read_csv(os.path.join(INPUT, "train.csv"))
test_data = pd.read_csv(os.path.join(INPUT, "test.csv"))
#公式アナウンスより修正
train_data.at[train_data['id']==2488,'judgement'] = 0
train_data.at[train_data['id']==7708,'judgement'] = 0

print(test_data.shape)
test_data.head()

# preprocess
train_data["combined_text"] = train_data["title"] + " " + train_data["abstract"].fillna("")
test_data["combined_text"] = test_data["title"] + " " + test_data["abstract"].fillna("")

(40834, 3)


In [26]:
label_cols = ['judgement']

# Config

In [10]:
# Parameters

BATCH_SIZE = 8
TEST_BATCH_SIZE = 8
NR_EPOCHS = 1
MAX_LEN = 300 # try diffrent lengths
threshold = 0.4
bert_model_name = './scibert_scivocab_uncased'
config = BertConfig.from_json_file('./scibert_scivocab_uncased/bert_config.json')

# Tokenizer

In [13]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)

def tokenize_sentences(sentences, tokenizer, max_seq_len = 128):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(sentence,                  
                                              add_special_tokens = True, 
                                              max_length = max_seq_len  
                                             )
        tokenized_sentences.append(tokenized_sentence)

    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        
        attention_masks.append(att_mask)

    return np.asarray(attention_masks)

input_ids = tokenize_sentences(train_data['combined_text'], tokenizer, MAX_LEN)
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
attention_masks = create_attention_masks(input_ids)

  0%|          | 0/27145 [00:00<?, ?it/s]

# SciBERT Classifier

In [14]:
class BertClassifier(tf.keras.Model):    
    
    def __init__(self, bert: TFBertModel, num_classes: int):
        super().__init__()
        self.bert = bert
        self.classifier = Dense(num_classes, activation='sigmoid')
        
    def call(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
        
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask)
        
        cls_output = outputs[1]
        
        cls_output = self.classifier(cls_output)
                
        return cls_output

In [15]:
def create_dataset(data_tuple, batch_size, train=True):
    
    dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
    
    dataset = dataset.repeat(1)
    
    if train:
        
        dataset = dataset.shuffle(buffer_size=10000)
    
    dataset = dataset.batch(batch_size)
    
    if train:
        
        dataset = dataset.prefetch(1)
        
    if not train:
        
        dataset = dataset.cache()
    
    return dataset

In [ ]:
def create_test_dataset():
    
    test_input_ids = tokenize_sentences(test_data['combined_text'], tokenizer, MAX_LEN)
    test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
    test_attention_masks = create_attention_masks(test_input_ids)
    test_steps = len(test_data) // TEST_BATCH_SIZE
    test_dataset = create_dataset((test_input_ids, test_attention_masks), batch_size=TEST_BATCH_SIZE, train=False)
    
    return test_dataset
    
    SUBMISSION_PATH = os.path.join(SUBMISSION, "basel.csv")

    submission = pd.read_csv(SUBMISSION_PATH, index_col='id')

def generate_class_probablities(model,test_dataset,test_steps):
    
    for i, (token_ids, masks) in enumerate(tqdm(test_dataset, total=test_steps)):
        
        sample_ids = test_data.iloc[i*TEST_BATCH_SIZE:(i+1)*TEST_BATCH_SIZE]['id']
        
        predictions = model(token_ids, attention_mask=masks).numpy()
        
        submission.loc[sample_ids, label_cols] = predictions
    
    return submission.loc[:,label_cols].values

test_dataset = create_test_dataset()

test_steps = len(test_data) // TEST_BATCH_SIZE

class_probs = np.zeros(shape =(len(test_data),len(label_cols)))

In [22]:
#Gradient Calculation

def train_step(model, token_ids, masks, labels):
    
    labels = tf.dtypes.cast(labels, tf.float32)
    with tf.GradientTape() as tape:
        predictions = model(token_ids, attention_mask=masks)
        loss = loss_object(labels, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables), name = 'gradients')
    
    train_loss(loss)

    for i, auc in enumerate(train_auc_metrics):
        
        auc.update_state(labels[:,i], predictions[:,i])
        
def validation_step(model, token_ids, masks, labels):
    
    labels = tf.dtypes.cast(labels, tf.float32)

    predictions = model(token_ids, attention_mask=masks, training=False)
    
    v_loss = loss_object(labels, predictions)

    validation_loss(v_loss)
    
    for i, auc in enumerate(validation_auc_metrics):
        
        auc.update_state(labels[:,i], predictions[:,i])   

# Model

In [ ]:
seeds = [0]

for seed in range(len(seeds)):
    
    print('=' * 50, f"CV {seed+1}", '=' * 50)
    model = BertClassifier(TFBertModel.from_pretrained(bert_model_name, from_pt=True, config = config), len(label_cols))
    labels =  train_data[label_cols].values
    train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=seed, test_size = 0.2)
    train_masks, validation_masks = train_test_split(attention_masks, random_state=seed, test_size=0.2)
    train_size = len(train_inputs)
    validation_size = len(validation_inputs)

    train_dataset = create_dataset((train_inputs, train_masks, train_labels), batch_size=BATCH_SIZE,train=True)
    validation_dataset = create_dataset((validation_inputs, validation_masks, validation_labels), batch_size=BATCH_SIZE,train=False)
    
    steps_per_epoch = train_size // (BATCH_SIZE)

    #  Loss Function
    loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=False)

    train_loss = tf.keras.metrics.Mean(name='train_loss')

    validation_loss = tf.keras.metrics.Mean(name='val_loss')

    #  Optimizer (with 1-cycle-policy)
    warmup_steps = steps_per_epoch // 3

    total_steps = steps_per_epoch * NR_EPOCHS - warmup_steps

    optimizer = create_optimizer(init_lr=2e-5, num_train_steps=total_steps, num_warmup_steps=warmup_steps)

    # Gradients
    
    gradients = 0
    
    #  Metrics
    train_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]

    validation_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]


    for epoch in range(NR_EPOCHS):

        print('=' * 50, f"EPOCH {epoch+1}", '=' * 50)

        start = time.time()


        for batch_no, (token_ids, masks, labels) in enumerate(tqdm(train_dataset)):

            train_step(model, token_ids, masks, labels)

            if batch_no % 100 == 0:

                    print(f'\nTrain Step: {batch_no}, Loss: {train_loss.result()}')

                    for i, label_name in enumerate(label_cols):

                        print(f"{label_name} roc_auc {train_auc_metrics[i].result()}")

                        train_auc_metrics[i].reset_states()

        for batch_no, (token_ids, masks, labels) in enumerate(tqdm(validation_dataset)):

            validation_step(model, token_ids, masks, labels)

        print(f'\nEpoch {epoch+1}, Validation Loss: {validation_loss.result()}, Time: {time.time()-start}\n')

        for i, label_name in enumerate(label_cols):

            print(f"{label_name} roc_auc {validation_auc_metrics[i].result()}")

            validation_auc_metrics[i].reset_states()

        print('\n')
        
    probs = generate_class_probablities(model,test_dataset,test_steps)
        
    submission.loc[:, label_cols] = probs
        
    submission.to_csv('probs'+str(seed)+'.csv')
        
    class_probs += probs

================================================== CV 1 ==================================================
================================================== EPOCH 1 ==================================================


  0%|          | 0/2715 [00:00<?, ?it/s]


Train Step: 0, Loss: 0.25303900241851807
judgement roc_auc 0.0

Train Step: 100, Loss: 0.21692171692848206
judgement roc_auc 0.4609745740890503

Train Step: 200, Loss: 0.1531851887702942
judgement roc_auc 0.5883769989013672

Train Step: 300, Loss: 0.13130639493465424
judgement roc_auc 0.6709978580474854

Train Step: 400, Loss: 0.12140591442584991
judgement roc_auc 0.7730823755264282

Train Step: 500, Loss: 0.11365943402051926
judgement roc_auc 0.7424203753471375

Train Step: 600, Loss: 0.10569839924573898
judgement roc_auc 0.8889495134353638

Train Step: 700, Loss: 0.10033294558525085
judgement roc_auc 0.90425705909729

Train Step: 800, Loss: 0.09676118940114975
judgement roc_auc 0.871980607509613

Train Step: 900, Loss: 0.09497271478176117
judgement roc_auc 0.9073358774185181


# function

In [ ]:
def opt_fbeta_threshold(y_true, y_pred):
    """fbeta score計算時のthresholdを最適化"""
    def opt_(x): 
        return -fbeta_score(y_true, y_pred >= x, beta=7)
    result = minimize(opt_, x0=np.array([0.1]), method='Powell')
    best_threshold = result['x'].item()
    return best_threshold


def metrics(y_true, y_pred):
    """fbeta(beta=7)の閾値最適化評価関数"""
    bt = opt_fbeta_threshold(y_true, y_pred)
    print(f"bt:{bt}")
    score = fbeta_score(y_true, y_pred >= bt, beta=7)
    return score


def skf(train, n_splits, random_state):
    """層化KFoldのインデックスのリストを作成"""
    skf = StratifiedKFold(n_splits=n_splits, random_state=random_state, shuffle=True)
    return list(skf.split(train, train[Config.target_col]))


def encode_texts(texts, tokenizer, max_length):
    """text encoding"""
    encoded_dict = tokenizer.batch_encode_plus(
        texts,
        padding = 'max_length',
        truncation = True,
        max_length = max_length,
    )
    return dict(encoded_dict)


def get_dataset(X, y=None, dataset="test"):
    """データをtf.Datasetの形式に変更"""
    if dataset=="train":
        train_dataset = (
            tf.data.Dataset
            .from_tensor_slices((X, y))
            .shuffle(2048)
            .batch(Config.train_batch_size)
            .prefetch(tf.data.experimental.AUTOTUNE)
            )
        if Config.steps_per_epochs is not None:
            train_dataset = train_dataset.repeat()

        return train_dataset

    elif dataset=="valid":
        valid_dataset = (
            tf.data.Dataset
            .from_tensor_slices((X, y))
            .batch(Config.valid_batch_size)
            .prefetch(tf.data.experimental.AUTOTUNE)
        )
        return valid_dataset
    
    elif dataset=="test":
        test_dataset = (
            tf.data.Dataset
            .from_tensor_slices(X)
            .batch(Config.test_batch_size)
            .prefetch(tf.data.experimental.AUTOTUNE)
        )
        return test_dataset


def get_model_and_tokenizer():
    """model&tokenizer を取得 (TPU ok)"""
    #
    if TPU:
        tf.config.experimental_connect_to_cluster(TPU)
        tf.tpu.experimental.initialize_tpu_system(TPU)
        tpu_strategy = tf.distribute.experimental.TPUStrategy(TPU)
        with tpu_strategy.scope():
            model = build_auto_model()
    else:
        model = build_auto_model()

    tokenizer = AutoTokenizer.from_pretrained(Config.model)
    return model, tokenizer


def get_class_weight(target, weight):
    """class weights の作成"""
    if weight == "balanced":
        class_weights = class_weight.compute_class_weight(
            class_weight='balanced',
            classes=np.unique(target),
            y=target)
        class_weights = dict(enumerate(class_weights))
    elif weight is None:
        class_weights = None
    else:
        # ex) weight = {0:0.2, 1:0.98}
        class_weights = weight
    
    return class_weights


def training_v1(train_df, valid_df, model, tokenizer, filepath):
    """training 用の関数"""
    #単語IDのシーケンス
    tr_text = encode_texts(texts=train_df[Config.text_col].tolist(), tokenizer=tokenizer, max_length=Config.max_length)
    va_text = encode_texts(texts=valid_df[Config.text_col].tolist(), tokenizer=tokenizer, max_length=Config.max_length)
    tr_dataset = get_dataset(X=tr_text, y=train_df[Config.target_col].values, dataset="train")
    va_dataset = get_dataset(X=va_text, y=valid_df[Config.target_col].values, dataset="valid")

    #学習の進捗状況を把握
    #CallBack：学習途中での結果を返す
    #ModelCheckpoint：最良のモデルを保存、保存するときの条件を下記で指定
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath, 
        moniter="val_loss", 
        verbose=1, 
        save_best_only=True, 
        save_weights_only=True,
        mode="min")
    
    #modelの学習、modelは（build_auto_model）で作っている?
    model.fit(tr_dataset, 
              epochs=Config.epochs, 
              verbose=1, 
              callbacks=[checkpoint],
              validation_data=va_dataset, 
              steps_per_epoch=Config.steps_per_epochs,
              class_weight=get_class_weight(train_df[Config.target_col], weight=Config.class_weight))


def inference_v1(test_df, model, tokenizer, filepath):
    """推論用の関数"""
    model.load_weights(filepath)
    #テキストの単語IDシーケンス化
    te_text = encode_texts(texts=test_df[Config.text_col].tolist(), tokenizer=tokenizer, max_length=Config.max_length)
    #tensorflowのデータ供給に関するライブラリ、GPUの待ち時間を減らしたり前処理パイプラインをまとめたり色々できるらしい
    te_dataset = get_dataset(X=te_text, y=None, dataset="test")

    preds = model.predict(te_dataset)
    return preds.reshape(-1)


def train_cv_v1(train, cv, metrics, name, dir):
    """cross validationの実行関数 (train)"""

    oof = np.zeros(len(train))
    for i_fold, (tr_idx, va_idx) in enumerate(cv):
        K.clear_session()
        #モデルとtokenizer(文書を単語分割をして、IDに変換するモジュール)をインスタンス化
        model, tokenizer = get_model_and_tokenizer()

        tr_df, va_df = train.iloc[tr_idx].reset_index(), train.iloc[va_idx].reset_index()
        filepath = os.path.join(dir, f"{name}_fold{i_fold+1}.h5")

        if not os.path.isfile(filepath):  # 学習済みモデルがあればtrainingしない
            training_v1(tr_df, va_df, model, tokenizer, filepath)
        
        #予測の実行
        preds = inference_v1(va_df, model, tokenizer, filepath)

        score = metrics(np.array(va_df[Config.target_col]), preds)
        #logger.info(f"{name}_fold{i_fold+1} >>> val socre:{score:.4f}")
        oof[va_idx] = preds
    
    score = metrics(np.array(train[Config.target_col]), oof)
    #logger.info(f"{name} >>> val score:{score:.4f}")
    return oof


def predict_cv_v1(test, name, dir):
    """cross validationの実行関数 (test)"""
    preds_fold = []
    for i_fold in range(Config.n_fold):
        model, tokenizer = get_model_and_tokenizer()

        filepath = os.path.join(dir, f"{name}_fold{i_fold+1}.h5")
        preds = inference_v1(test, model, tokenizer, filepath)
        preds_fold.append(preds)

        #logger.info(f"{name}_fold{i_fold+1} inference")
    
    preds = np.mean(preds_fold, axis=0)
    return preds


def visualize_confusion_matrix(
        y_true,
        pred_label,
        height=.6,
        labels=None):
    """混合行列をプロット 
    (https://www.guruguru.science/competitions/11/discussions/2fb11851-67d0-4e96-a4b1-5629b944f363/)"""
    
    conf = confusion_matrix(y_true=y_true,
                            y_pred=pred_label,
                            normalize='true')

    n_labels = len(conf)
    size = n_labels * height
    fig, ax = plt.subplots(figsize=(size * 4, size * 3))
    sns.heatmap(conf, cmap='Blues', ax=ax, annot=True, fmt='.2f')
    ax.set_ylabel('Label')
    ax.set_xlabel('Predict')

    if labels is not None:
        ax.set_yticklabels(labels)
        ax.set_xticklabels(labels)
        ax.tick_params('y', labelrotation=0)
        ax.tick_params('x', labelrotation=90)

    plt.show()
    return fig

# Main

In [ ]:
def main():
    # load data
    train = pd.read_csv(os.path.join(INPUT, "train.csv"))
    test = pd.read_csv(os.path.join(INPUT, "test.csv"))
    #公式アナウンスより修正
    train.at[train['id']==2488,'judgement'] = 0
    train.at[train['id']==7708,'judgement'] = 0

    if Config.debug:
        train = train.sample(1000, random_state=Config.seeds[0]).reset_index(drop=True)
        test = test.sample(1000, random_state=Config.seeds[0]).reset_index(drop=True)


    # training
    oof_df = pd.DataFrame()
    for seed in Config.seeds:
        name = f"{Config.name_v1}-{seed}"
        #cross validationの実行関数
          #skf 層化KFoldのインデックスのリストを作成
          #metrics fbeta(beta=7)の閾値最適化評価関数
        oof = train_cv_v1(train, 
                        cv=skf(train, n_splits=Config.n_fold, random_state=seed),
                        metrics=metrics, 
                        name=name, 
                        dir=OUTPUT_EXP_MODEL)
        oof_df[name] = oof

    oof_df.to_csv(os.path.join(OUTPUT_EXP_PREDS, "oof.csv"), index=False)  # oof 予測値(prob)を保存

    y_true = train[Config.target_col]
    y_pred = oof_df.mean(axis=1)

    best_threshold = opt_fbeta_threshold(y_true.values, y_pred.values)
    oof_score = fbeta_score(y_true, y_pred >= best_threshold, beta=7)
    comments = f"score:{oof_score:.4f}/threshold:{best_threshold}"  # 最終的なスコアと閾値 (スペースがあるとsubmit時のコメントエラーが出る)
    #logger.info(comments)

    fig = visualize_confusion_matrix(y_true, y_pred>=best_threshold)  # 混合行列を表示&save
    fig.savefig(os.path.join(OUTPUT_EXP_FIG, "cm.png"), dpi=300)

    
    preds_df = pd.DataFrame()
    for seed in Config.seeds:
        name = f"{Config.name_v1}-{seed}"
        preds = predict_cv_v1(test, name, dir=OUTPUT_EXP_MODEL)
        preds_df[name] = preds
        
    # test予測値(prob)を保存
    preds_df.to_csv(os.path.join(OUTPUT_EXP_PREDS, "preds.csv"), index=False)  

    # submit
    if Config.submit:
        sub_df = pd.read_csv(os.path.join(INPUT, "sample_submit.csv"), header=None)
        sub_df.columns = ["id", "judgement"]

        preds = preds_df.mean(axis=1)
        sub_df["judgement"] = (preds.values >= best_threshold) * 1
        
        filepath = os.path.join(SUBMISSION, f"{EXP}.csv")
        sub_df.to_csv(filepath, index=False, header=False)
        
        # signate api を使ってsubmit
        ! signate submit --competition-id=471 $filepath --note $comments